In [351]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import scale

In [353]:
# get ELA scores
dfELA = pd.read_csv('ELA_Scores.csv')
print(dfELA.shape)
dfELA.head()

(3652890, 18)


,School Year,State,NCES LEA ID,LEA,School,NCES SCH ID,Data Group,Data Description,Value,Numerator,Denominator,Population,Subgroup,Characteristics,Age/Grade,Academic Subject,Outcome,Program Type
0,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,584,Performance on Statewide Reading/Language Arts...,42%,NaN,881,All Students,All Students in School,NaN,All Grades,Reading/Language Arts,Percent Proficient,NaN
1,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,584,Performance on Statewide Reading/Language Arts...,S,NaN,2,All Students,American Indian/Alaska Native/Native American,NaN,All Grades,Reading/Language Arts,Percent Proficient,NaN
2,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,584,Performance on Statewide Reading/Language Arts...,S,NaN,3,All Students,Asian/Pacific Islander,NaN,All Grades,Reading/Language Arts,Percent Proficient,NaN
3,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,584,Performance on Statewide Reading/Language Arts...,21-39%,NaN,30,All Students,Black (not Hispanic) African American,NaN,All Grades,Reading/Language Arts,Percent Proficient,NaN
4,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,584,Performance on Statewide Reading/Language Arts...,31%,NaN,462,All Students,Hispanic/Latino,NaN,All Grades,Reading/Language Arts,Percent Proficient,NaN


In [354]:
# get only schoolwide videos
dfELA = dfELA.loc[(dfELA['Age/Grade'] == 'All Grades') & (dfELA['Subgroup'] == 'All Students in School') ]
dfELA = dfELA[['NCES LEA ID', 'LEA', 'School', 'NCES SCH ID', 'Value']]
print(dfELA.shape)
dfELA.head()

(77573, 5)


,NCES LEA ID,LEA,School,NCES SCH ID,Value
0,100005,Albertville City,Albertville Middle School,10000500870,42%
47,100005,Albertville City,Albertville High School,10000500871,22%
73,100005,Albertville City,Albertville Intermediate School,10000500879,39%
120,100005,Albertville City,Albertville Elementary School,10000500889,42%
167,100006,Marshall County,Kate Duncan Smith DAR Middle,10000600193,54%


In [355]:
# get math scores
dfMath = pd.read_csv('Math_scores.csv')
print(dfMath.shape)
dfMath.head()

(3935036, 18)


,School Year,State,NCES LEA ID,LEA,School,NCES SCH ID,Data Group,Data Description,Value,Numerator,Denominator,Population,Subgroup,Characteristics,Age/Grade,Academic Subject,Outcome,Program Type
0,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,583,Performance on Statewide Mathematics Assessment,11%,NaN,884,All Students,All Students in School,NaN,All Grades,Mathematics,Percent Proficient,NaN
1,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,583,Performance on Statewide Mathematics Assessment,S,NaN,2,All Students,American Indian/Alaska Native/Native American,NaN,All Grades,Mathematics,Percent Proficient,NaN
2,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,583,Performance on Statewide Mathematics Assessment,S,NaN,4,All Students,Asian/Pacific Islander,NaN,All Grades,Mathematics,Percent Proficient,NaN
3,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,583,Performance on Statewide Mathematics Assessment,<=10%,NaN,32,All Students,Black (not Hispanic) African American,NaN,All Grades,Mathematics,Percent Proficient,NaN
4,2020-2021,ALABAMA,100005,Albertville City,Albertville Middle School,10000500870,583,Performance on Statewide Mathematics Assessment,9%,NaN,465,All Students,Hispanic/Latino,NaN,All Grades,Mathematics,Percent Proficient,NaN


In [356]:
# get only schoolwide videos
dfMath = dfMath.loc[(dfMath['Age/Grade'] == 'All Grades') & (dfMath['Subgroup'] == 'All Students in School')]
dfMath = dfMath[['NCES LEA ID', 'LEA', 'School', 'NCES SCH ID', 'Value']]
print(dfMath.shape)
dfMath.head()

(82303, 5)


,NCES LEA ID,LEA,School,NCES SCH ID,Value
0,100005,Albertville City,Albertville Middle School,10000500870,11%
47,100005,Albertville City,Albertville High School,10000500871,19%
73,100005,Albertville City,Albertville Intermediate School,10000500879,16%
120,100005,Albertville City,Albertville Elementary School,10000500889,25%
167,100006,Marshall County,Kate Duncan Smith DAR Middle,10000600193,13%


In [357]:
# merge math and ELA scores together
dfScores = dfMath.merge(dfELA, how = 'inner', on = 'NCES SCH ID')
dfScores = dfScores.rename({'Value_x':'Math_scores', 'Value_y':'ELA_scores'}, axis = 1)
dfScores = dfScores[['NCES SCH ID', 'Math_scores', 'ELA_scores']]
dfScores = dfScores.loc[(dfScores['Math_scores'] != 'S') & (dfScores['ELA_scores'] != 'S')]
dfScores.head()

,NCES SCH ID,Math_scores,ELA_scores
0,10000500870,11%,42%
1,10000500871,19%,22%
2,10000500879,16%,39%
3,10000500889,25%,42%
4,10000600193,13%,54%


In [358]:
# clean up percent columns
def percent_to_float(value):
    if not isinstance(value, str):
        return float('nan')  # or handle as you prefer
        
    value = value.strip()
    
    # Handle special cases first
    if value == 'S':
        return float('nan')  # or some other value
    
    # Remove percentage signs
    value = value.replace('%', '')
    
    # Handle inequalities
    if value.startswith('>='):
        return float(value[2:]) / 100
    elif value.startswith('<='):
        return float(value[2:]) / 100
    elif value.startswith('<'):
        return float(value[1:]) / 100
    elif value.startswith('>'):
        return float(value[1:]) / 100
    
    # Handle ranges
    if '-' in value:
        parts = value.split('-')
        try:
            low = float(parts[0])
            high = float(parts[1])
            return ((low + high) / 2) / 100
        except:
            return float('nan')
    
    # Handle simple percentages
    try:
        return float(value) / 100
    except:
        return float('nan')

In [359]:
# clean up df
for i in list(dfScores.index):
    dfScores.loc[i, 'Math_scores'] = percent_to_float(dfScores.loc[i, 'Math_scores'])
    dfScores.loc[i, 'ELA_scores'] = percent_to_float(dfScores.loc[i, 'ELA_scores'])

dfScores['Math_scores'] = dfScores['Math_scores'].astype('float')
dfScores['ELA_scores'] = dfScores['ELA_scores'].astype('float')
dfScores = dfScores.dropna()
dfScores.dtypes

NCES SCH ID      int64
Math_scores    float64
ELA_scores     float64
dtype: object

In [360]:
# convert to z-score
dfScores['Math_scores'] = scale(dfScores['Math_scores'])
dfScores['ELA_scores'] = scale(dfScores['ELA_scores'])
print(dfScores.shape)
dfScores.head()

(68606, 3)


,NCES SCH ID,Math_scores,ELA_scores
0,10000500870,-1.190436,-0.076501
1,10000500871,-0.802871,-1.091397
2,10000500879,-0.948208,-0.228735
3,10000500889,-0.512198,-0.076501
4,10000600193,-1.093545,0.532437


In [361]:
# add together scores to create index for school performance
dfScores['Agg_Score_Index'] = dfScores['Math_scores'] + dfScores['ELA_scores']
dfScores = dfScores[['NCES SCH ID', 'Agg_Score_Index']]
print(dfScores.shape)
dfScores.head()

(68606, 2)


,NCES SCH ID,Agg_Score_Index
0,10000500870,-1.266936
1,10000500871,-1.894269
2,10000500879,-1.176943
3,10000500889,-0.588699
4,10000600193,-0.561107


In [362]:
# read in school geodata
schools = pd.read_excel('EDGE_GEOCODE_PUBLICSCH_2324/EDGE_GEOCODE_PUBLICSCH_2324.xlsx')
schools = schools[['NCESSCH', 'LON', 'LAT']]
print(schools.shape)
schools.head()

(102274, 3)


,NCESSCH,LON,LAT
0,10000500870,-86.206200,34.2602
1,10000500871,-86.204900,34.2622
2,10000500879,-86.220100,34.2733
3,10000500889,-86.221806,34.2527
4,10000501616,-86.193300,34.2898


In [363]:
# merge school and score data
dfScores = dfScores.merge(schools, how = 'inner', left_on = 'NCES SCH ID', right_on = 'NCESSCH')
dfScores = dfScores[['Agg_Score_Index', 'LON', 'LAT']]
print(dfScores.shape)
dfScores.head()

(67813, 3)


,Agg_Score_Index,LON,LAT
0,-1.266936,-86.206200,34.26020
1,-1.894269,-86.204900,34.26220
2,-1.176943,-86.220100,34.27330
3,-0.588699,-86.221806,34.25270
4,-0.561107,-86.254153,34.53375


In [364]:
# make our lon and lat into a point type for spacial join
dfScores['geometry'] = dfScores.apply(lambda row: Point(row['LON'], row['LAT']), axis=1)
dfScores = dfScores[['Agg_Score_Index', 'geometry']]
dfScores.head()

,Agg_Score_Index,geometry
0,-1.266936,POINT (-86.2062 34.2602)
1,-1.894269,POINT (-86.20489999999999 34.2622)
2,-1.176943,POINT (-86.2201 34.2733)
3,-0.588699,POINT (-86.221806 34.2527)
4,-0.561107,POINT (-86.254153 34.53375)


In [365]:
# Read shapefile (automatically loads .shp, .shx, .dbf)
gdf = gpd.read_file("cb_2022_us_bg_500k/cb_2022_us_bg_500k.shp")

# View the first 5 rows
print(gdf.shape)
gdf.head()

(242297, 12)


,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geometry
0,04,019,940800,2,1500000US040199408002,040199408002,2,Block Group 2,BG,2033911373,68536,"POLYGON ((-112.41 31.857, -112.38 31.859, -112..."
1,04,003,000901,4,1500000US040030009014,040030009014,4,Block Group 4,BG,486316,0,"POLYGON ((-109.55 31.334, -109.55 31.339, -109..."
2,06,073,018102,3,1500000US060730181023,060730181023,3,Block Group 3,BG,324091,3579,"POLYGON ((-117.37 33.182, -117.37 33.182, -117..."
3,06,073,007200,3,1500000US060730072003,060730072003,3,Block Group 3,BG,720442,709604,"POLYGON ((-117.26 32.725, -117.26 32.726, -117..."
4,06,073,017702,3,1500000US060730177023,060730177023,3,Block Group 3,BG,317237,0,"POLYGON ((-117.3 33.052, -117.3 33.053, -117.3..."


In [366]:
# make sure points match the crs of gdf
gdfPoints = gpd.GeoDataFrame(
    dfScores, 
    geometry='geometry',
    crs="EPSG:4269"  # WGS84 coordinate reference system
)
gdfPoints.head()

,Agg_Score_Index,geometry
0,-1.266936,POINT (-86.206 34.26)
1,-1.894269,POINT (-86.205 34.262)
2,-1.176943,POINT (-86.22 34.273)
3,-0.588699,POINT (-86.222 34.253)
4,-0.561107,POINT (-86.254 34.534)


In [367]:
# Basic spatial join (points within polygons)
result = gpd.sjoin(
    gdfPoints, 
    gdf,
    how="left",       # 'left', 'right', or 'inner'
    predicate="within" # Spatial relationship
)

In [368]:
print(result.shape)
result.head()

(67813, 14)


,Agg_Score_Index,geometry,index_right,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER
0,-1.266936,POINT (-86.206 34.26),62691,01,095,030902,1,1500000US010950309021,010950309021,1,Block Group 1,BG,956528,0
1,-1.894269,POINT (-86.205 34.262),62691,01,095,030902,1,1500000US010950309021,010950309021,1,Block Group 1,BG,956528,0
2,-1.176943,POINT (-86.22 34.273),53551,01,095,030903,3,1500000US010950309033,010950309033,3,Block Group 3,BG,3593756,0
3,-0.588699,POINT (-86.222 34.253),191141,01,095,030902,2,1500000US010950309022,010950309022,2,Block Group 2,BG,14532780,71529
4,-0.561107,POINT (-86.254 34.534),211006,01,095,030205,1,1500000US010950302051,010950302051,1,Block Group 1,BG,72885330,868079


In [369]:
# get aggregate exam score
scores = result.groupby('AFFGEOID', as_index=False)['Agg_Score_Index'].mean()
print(scores.shape)
scores.head()

(49436, 2)


,AFFGEOID,Agg_Score_Index
0,1500000US010010202001,0.075583
1,1500000US010010205011,0.857609
2,1500000US010010207001,-0.248511
3,1500000US010010208041,-0.300325
4,1500000US010010209011,-1.363828


In [370]:
scores.isna().sum()

AFFGEOID           0
Agg_Score_Index    0
dtype: int64

In [371]:
# export as a csv
scores.to_csv('Block_Group_Exam_Score.csv', index=False)